# Algorítmo con mayor progreso (al 2024 04 02) para calcular los valores nan del censo.

## Explicación del método:

La base del método (usar minimize de scipy.optimize) fue sugerencia de chat gpt, se recomienda re-explorar opciones.

Este notebook tiene el método más desarrollado hasta el 2024 04 02 para calcular los nans del censo.

__INPUT:__
Se generó un caso simplificado en el que se asume que hay 5 manzanas con las columnas P_0A2, P_0A2_F y P_0A2_M
   * 1 manzana con todos los datos conocidos
   * 3 manzanas con 1 solo nan
   * 1 manzana con todos los datos desconocidos
Aparte se conocen los valores totales del AGEB

In [1]:
# Caso simplificado (Unknowns en paréntesis)

# FILAS (Manzanas)
# P_0A2_tot | P_0A2_F | P_0A2_M
#     4     |    4    |    0
#    nan0   |    5    |    2
#     3     |   nan1  |    2
#     4     |    3    |   nan2
#    nan3   |   nan4  |   nan5

# TOTALES (AGEB)
# P_0A2_tot | P_0A2_F | P_0A2_M
#     28    |   18    | 10

# Respuestas esperadas por parte del algorítmo
# nan0 --> 7
# nan1 --> 1
# nan2 --> 1
# nan3 --> 10
# nan4 --> 5
# nan5 --> 5

__PROCESO:__
El proceso de resolución usa minimize de scipy.optimize. 

resultado = minimize(objective_function, 
                         initial_guess, 
                         args=(block_values,),
                         constraints=constraints,
                         bounds=[(0, None)] * (unknown_values))


* La __ecuación objetivo__ (objective_function) es lo que se busca minimizar (En este caso, la cantidad de nans que hay en el gdf de manzanas).
* El __initial_guess__ es un array del tamaño de la cantidad de valores desconocidos. (Si hay 6 nans, un array de 6 valores iniciales). Se eligieron 0s.
* Los __args__ corresponden a los argumentos de la objective_function
* Los __constraints__ son lo complicado, y lo que más tomó tiempo:

In [2]:
# Explicación basica de las constraints
#Las constraints se agregan en la función minimize como lista de diccionarios.
    
#constraint = {'type': 'eq', 
#              'fun': computable_equation, 
#              'args': ()}

# En cada diccionario de restricciones, 'type':'eq' significa que la restricción es de tipo igualdad. (La computable_equation debe dar igual a cero)
# La computable_equation de 'fun' debe ser una función que reciba los argumentos 'args' y de igual a cero.
# En esa computable_equation deben estar las variables desconocidas (representadas como x, el array del initial_guess, y su posición --> x[i]
    
# Por ejemplo, sabemos que P_0A2 = P_0A2_F + P_0A2_M., su la función de igualdad se ve así: P_0A2 - (P_0A2_F + P_0A2_M) #Porque esto debe dar igual a cero.
# Por lo tanto, específicamente para la segunda fila del caso simplificado, la computable_equation debería ser: x[0] - (5 + 2)
# Es x[0] porque es el nan0. Cada nan debe tener un i unico (Es una variable desconocida única).

# A este tipo de constraints se les llamó row_constraints. También hay column_constraints.
# Por ejemplo, sabemos que P_0A2 del ageb = la suma de P_0A2 de las manzanas, su la función de igualdad se ve así: 
# P_0A2_ageb - (Valores conocidos de P_0A2 de manzanas + Valores desconocidos [x] de P_0A2 de las manzanas)

Las funciones create_row_constraints y create_col_constraints se dedican a crear por primera vez el diccionario para cada constraint (Lo más importante es la función 'fun' y los argumentos 'args' para cada nan value encontrado.  Las funciones row_computable_equation y col_computable_equation son las computable_equations que van dentro del diccionario de constraints. Es decir, las primeras dos funciones crean la función y argumentos que minimize de  scipy.optimize va a estar utilizando para encontrar las variables desconocidas x[i].

__PENDIENTE:__ Hacer que solo sea necesario recibir una ecuación para que haga lo necesario, por ahora está hard-coded a P_0A2 = P_0A2_F + P_0A2_M).

__OBSTÁCULOS:__
Hasta ahora el mayor obstáculo es que el caso simplificado no puede ser resuelto porque hay 7 constraints (4 de rows y 3 de columnas) y solo 6 variables. Por lo tanto regresa el mensaje "More equality constraints than independent variables". (Tragedia).

# Notebook code:

## Import libraries

In [3]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from scipy.optimize import minimize
import shapely

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:25: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Etapa 1: Resolver para un caso simplificado [Not solved]

### Creación del caso simplificado

In [4]:
blocks_values_simplified = pd.DataFrame( {'P_0A2_tot': [4, None, 3, 4,None],
                                          'P_0A2_F': [4, 5, None, 3,None],
                                          'P_0A2_M': [0, 2, 2, None,None]})
blocks_values_simplified

,P_0A2_tot,P_0A2_F,P_0A2_M
0,4.0,4.0,0.0
1,NaN,5.0,2.0
2,3.0,NaN,2.0
3,4.0,3.0,NaN
4,NaN,NaN,NaN


In [5]:
ageb_values_simplified = pd.DataFrame( {'P_0A2_tot': [28],
                                        'P_0A2_F': [18],
                                        'P_0A2_M': [10]})
ageb_values_simplified

,P_0A2_tot,P_0A2_F,P_0A2_M
0,28,18,10


### Constraints para rows

In [27]:
def row_computable_equation(x,row_index,row_vals,nan_i):
    # This is an computable/iterable equation, will repeat itself inside minimize from scipy.optimice in order to find unknowns.
    # What's most important (what makes it unique for each case constraint) are the arguments created on function create_row_constraints.

    ### INPUTS:
    ### x --> Array of unknowns for the whole system (Where guesses of unknown values are stored).
    ### row_index --> Current row index (Used to remember where an unknown was found).
    ### row_vals --> Real, original values of row's knowns and unknowns (nans).
    ### nan_i --> Number of first nan identified in current row.

    ### OUTPUT:
    ### computable/iterable equation for a given row that contains known values for that row and unknowns in the form of array[id] --> x[nan_i].

    ###################################################################################################################
    ####################### PENDIENTE: 
    ####################### Esto debe ser un argumento
    current_equation = 'P_0A2_tot - (P_0A2_F + P_0A2_M)'
    ###################################################################################################################
    
    # Get global variable constrained_cols: a dictionary containing all nan_is found in each column (col:[nan_i,nan_i,...])
    # If a nan value is found on a column, it gets registered on this dictionary for later use in columns_constraints function.
    global constrained_cols

    # Renaming row values (original values: value, nan) as equation variables
    # (eq_vars differenciates between known and unknown variables using x[i] instead of nan.)
    eq_vars = row_vals.copy()
    
    # Renaming current equation (original str of equation) as fixed equation.
    # (fixed_equation is part of the output, it will store current row's knowns and unknowns instead of col names.)
    fixed_equation = current_equation

    # Log info (Counts nans found on current row)
    initial_nan_id = nan_i
    print(f"Reading row {row_index}'s equation {current_equation} with original vals {row_vals}.")

    # Extract col_names used in equation
    col_names_lst = current_equation
    delimiters = [" ", "+", "-", "(",")"]
    for delimiter in delimiters:
        col_names_lst = " ".join(col_names_lst.split(delimiter))
    col_names_lst = col_names_lst.split()
    
    # Find where there are nan values
    # The position is given according to position of values in equation. For example, if equation is P_0A2_tot - (P_0A2_F + P_0A2_M):
    # ---> eq_vars will be in that order (P_0A2_tot value, P_0A2_F value, P_0A2_M value).
    # ---> If values in those eq_vars are (nan, value, nan), row_nans_position will be [0,2].
    # Therefore, it works for any number of variables in any given equation.
    row_nans_position, = np.where(np.isnan(eq_vars))
    
    # For each variable position
    for position in range(len(eq_vars)):
        # Column name being examined in current row 
        # (Since both eq_vars's position and variable's position come from position in equation)
        col_name = col_names_lst[position]
        
        # If variable position was detected as nan
        if position in row_nans_position:

            ###################################################################################################################
            ####################### PENDIENTE: Esto fue diseñado sin tener en cuenta varias ecuaciones. ¿Que pasa si una ecuación usa P_0A2 y luego otra ecuación vuelve a usar P_0A2?
            #######################            Lo que va a pasar actualmente es que se va a re-registrar ese nan con un nuevo (mayor) nan_id.
            #######################            Lo que debería pasar es que 1. Revise si el nan ya está registrado en un único diccionario de nans_location con {col:{row:nan_id}} 
            #######################            (Try) Si ya está registrado, tomar ese nan_id y sustituir en la fixed_equation por ese nan_id y no aumentar el global nan_id.
            #######################            (Except) Si no está registrado, registrarlo, sustituir la fixed_equation y sí aumentar el global nan_id
            ###################################################################################################################
            
            # Replace col name in equation string with unknown variable in the form of x[current nan_i]
            fixed_equation = fixed_equation.replace(col_name,f'x[{nan_i}]')
            
            # Register current nan_i to corresponding column in constrained_cols dictionary
            try:
                # If key already exists, check nan_is registered
                col_nans = constrained_cols[col_name]
                if nan_i in col_nans: # If nan_i is already registered in this key (column name), pass.
                    pass
                else: # Else, register in this key (column name)
                    col_nans.append(nan_i)
                    constrained_cols[col_name] = col_nans
            except:
                # If key does not exist, create key and list with first nan_i
                constrained_cols[col_name] = [nan_i]

            # Write current unknown variable's location (row_index and column) in dicc to know which value to replace value at the very end
            global nans_location
            nans_location[nan_i] = (row_index,col_name)
            
            # Increment found nans (i)
            nan_i += 1

        # Else (variable position was NOT detected as nan), value is known.
        else:
            # Replace variable name in equation with known value
            fixed_equation = fixed_equation.replace(col_name, f"{eq_vars[position]}")

    if extended_logs:
        final_nan_id = nan_i
        nan_ids_found = final_nan_id - initial_nan_id
        
        print(f"Row {row_index} Step 3: Created computable row equation: {fixed_equation} by replacing {nan_ids_found} nans as x[i] unknowns.")
        print(f"Row {row_index} Step 3: Finishing row with nan_i: {nan_i}.")

    return eval(fixed_equation)

In [28]:
def create_row_constraints(blocks_values_simplified, nan_i=0):
    # This function happens just once, to create row_constraints in the form of dicctionary.

    # EXPLICACIÓN DE LAS CONSTRAINTS: Las constraints se agregan en la función minimize como lista de diccionarios.
    # En cada diccionario de restricciones, 'type':'eq' significa que la restricción es de tipo igualdad. 
    # Esto indica que queremos que una función de igualdad (definida en 'fun') sea igual a cero.
    # La función de igualdad que se debe colocar en 'fun' se crea en las siguientes definiciones. Lo que va después del return debe ser igual a cero.

    ### INPUTS:
    ### blocks --> gdf where unknown values (nans) are located
    ### nan_i --> Optional, number of first nan (Starting value normaly 0)

    ### OUTPUT:
    ### row_constraints dictionary (all row constraints) in the form of:
    ### constraints_dicc = {'type': 'eq', 
    ###                     'fun': computable_equation, 
    ###                     'args': ()}

    ###################################################################################################################
    ####################### PENDIENTE: 
    ####################### Aquí debería recibir también como argumento una lista de ecuaciones en string de las cuales se van a extraer las variables por cada ecuación)
    variables = ['P_0A2_tot','P_0A2_F','P_0A2_M']
    ###################################################################################################################
    
    row_constraints = []
    
    #----- RESTRICCIONES QUE SON POR FILA ---------------------------------------------------------------------------------------------------------------------
    # En este caso (Hard-coded por ahora) la restricción debería asegurar que por fila (por .iterrows) P_0A2 sea igual a P_0A2_F + P_0A2_M.
    # El número de variables encontradas (i) corresponderá al número de nans que hay en cada row.
    
    for row_index, row in blocks_values_simplified.iterrows():
        
        # GENERAL ROW-LOOP STRUCTURE:
        # STEP 1: Get current row's values - Obtains current row values (known values and unknown variables nans)
        # STEP 2: Evaluate nan quantity (If there's no nans in row, continue to next row as generating constrains without unknown variables is not usefull)
        # STEP 3: Create equation given current data (Inside row_computable_equation function, nans get substituded as x[nan_i], computable variables)
        # STEP 4: return constraint (dicctionary type)
        
        print('--'*20)
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 1: Get current row's values --------------------------------------------------------------------------------------------------------------------
        row_vals = []
        for col in variables:
            row_vals.append(row[col])
        # Print analysis logs
        if extended_logs:
            print(f"Row {row_index} Step 1: Variables: {variables} Values: {row_vals}.")

        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 2: Evaluate nan quantity -----------------------------------------------------------------------------------------------------------------------
        nans_found = len(np.where(np.isnan(row_vals))[0])
        if nans_found == 0:
            if extended_logs:
                print(f"Row {row_index} Step 2: Skipped as it has no nan values in currently examined columns.")
            continue
        else:
            if extended_logs:
                print(f"Row {row_index} Step 2: Nans found. Proceeding to creating constraint equation with initial nan_id {nan_i}.")

        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 3: Create equation given current data ----------------------------------------------------------------------------------------------------------
        # Constraint function for the current row and current columns being evaluated. 
        # This function will be analysed several times inside optimize.minimize using 'args' provided in the constraint dictionary. (x is provided by optimize.minimize)
         
        ###################################################################################################################
        ####################### PENDIENTE: 
        ####################### Pasarle la ecuación original a row_computable_equation
        ###################################################################################################################
        
        fixed_equation = row_computable_equation(x,row_index,row_vals,nan_i)
        if extended_logs:
            print(f"Row {row_index} Step 3: Finished creating constraint equation.") #for ecuation {ecuation} cuando lo tengamos escrito.
        
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 4: Return constraint ---------------------------------------------------------------------------------------------------------------------------
        current_row_dicc = {'type': 'eq', 
                            'fun': row_computable_equation, 
                            'args': (row_index,row_vals,nan_i)}
        row_constraints.append(current_row_dicc)
        # Print analysis logs
        if extended_logs:
            print(f"Row {row_index} Step 4: Constraints dicc: {current_row_dicc}.")

        # Update nan_i with nans that will be substituded inside current_row_eq_args()
        
        ###################################################################################################################
        ####################### PENDIENTE: Esto fue diseñado sin tener en cuenta varias ecuaciones. ¿Que pasa si una ecuación usa x[5] y luego otra usa x[5]
        #######################            Lo que va a pasar actualmente es que, aunque no se haya usado un nuevo nan_id, se va a aumentar el global nan_id
        #######################            de acuerdo con la cantidad de nans de la fila en lugar de la cantidad de nuevos nan_ids de x[i] agregados.
        ###################################################################################################################
        nan_i = nan_i + nans_found
        
        # Print analysis logs
        if extended_logs:
            print(f"Finished row. Updated global nan_i to {nan_i}.")
            
    return row_constraints, nan_i

### Constraints para columns

In [29]:
def create_col_constraints(blocks_values_simplified, ageb_values_simplified, constrained_cols):
    # This function happens just once, to create row_constraints in the form of dicctionary.

    # EXPLICACIÓN DE LAS CONSTRAINTS: Las constraints se agregan en la función minimize como lista de diccionarios.
    # En cada diccionario de restricciones, 'type':'eq' significa que la restricción es de tipo igualdad. 
    # Esto indica que queremos que una función de igualdad (definida en 'fun') sea igual a cero.
    # La función de igualdad que se debe colocar en 'fun' se crea en las siguientes definiciones. Lo que va después del return debe ser igual a cero.

    ### INPUTS:
    ### blocks --> gdf where unknown values (nans) are located
    ### ageb --> gdf where total known values of blocks are located
    ### constrained_cols --> dictionary feed while looping through row_computable_equation that contains all unique nan_ids registered for each column
    
    ###################################################################################################################
    ####################### PENDIENTE: Si se va a hacer un único diccionario de nans_location con {col:{row:nan_id}}, todo lo de constrained_cols debe reajustarse.
    ###################################################################################################################

    ### OUTPUT:
    ### row_constraints dictionary (all cols constraints) in the form of:
    ### constraints_dicc = {'type': 'eq', 
    ###                     'fun': computable_equation, 
    ###                     'args': ()}
    
    #----- RESTRICCIONES QUE SON POR COLUMNA  -----------------------------------------------------------------------------------------------------------------
    # En este caso (Hard-coded por ahora), la restricción debería asegurar que por fila (por .iterrows) P_0A2 sea igual a P_0A2_F + P_0A2_M.
    # El número de variables encontradas (i) corresponderá al número de nans que hay en cada row.
    # La restricción para cualquier columna (debe ser igual a cero) es:
    #--> La suma de los valores actuales (conocidos) en la columna del gdf de manzanas (+) los valores (desconocidos/hallados) de x[] hasta ahora (-) El valor total (conocido) por AGEB
    
    col_constraints = []
    for col_name in constrained_cols.keys():

        # GENERAL COLUMN-LOOP STRUCTURE:
        # STEP 1: Get col value and nan_id_unknowns found on that col- Obtains current total known col value (AGEB total) and unknowns found on that col (nan_i_unknowns)
        # STEP 2: Evaluate nan quantity (If there's no nans in row, continue to next row as generating constrains without unknown variables is not usefull)
        # STEP 3: Create equation given current data (Inside row_computable_equation function, nans get substituded as x[nan_i], computable variables)
        # STEP 4: return constraint (dicctionary type)
        
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 1: Get col value and nan_id_unknowns found on that col -----------------------------------------------------------------------------------------
        # AGEB Value
        col_val = ageb_values_simplified[col_name].unique()[0]
        # Unknowns found while looping through rows
        nan_i_unknowns = list(set(constrained_cols[col_name]))

        # Print analysis logs
        if extended_logs:
            print(col_name)
            print(nan_i_unknowns)
            print(f"Column {col_name} Step 1: AGEB total value: {col_val}.")
            
        #------------------------------------------------------------------------------------------------------------------------------------------------------
        # STEP 2: Set col args (blocks_values, unknown_col_vars, total_value) ---------------------------------------------------------------------------------
        
        def col_computable_equation(x,blocks_values_simplified,nan_i_unknowns,col_name,col_val):
            # This is an computable/iterable equation, will repeat itself inside minimize from scipy.optimice in order to find unknowns.
            # What's most important (what makes it unique for each case constraint) are the arguments created on function create_col_constraints.
        
            ### INPUTS:
            ### x --> Array of unknowns for the whole system (Where guesses of unknown values are stored).
            ### blocks --> gdf where unknown values (nans) are located
            ### nan_i_unknowns --> list of unique nan_ids located in current column
            ### col_name --> Current column name
            ### col_val --> Total AGEB value for current col
        
            ### OUTPUT:
            ### computable/iterable equation for a given row that contains known values for that row and unknowns in the form of array[id] --> x[nan_i].

            print(f"Verification log - Reading function for col {col_name}.")
            return np.nansum(blocks_values_simplified[col_name]) + np.nansum(x[nan_i_unknowns]) - col_val

        col_constraints.append({'type': 'eq', 
                                'fun': col_computable_equation,
                                'args':(blocks_values_simplified,nan_i_unknowns,col_name,col_val)})

    return col_constraints

### Objective function

In [31]:
# Definir la función objetivo (Lo que queremos minimizar con la función minimize): la cantidad de NaNs.
def objective_function(x, block_values):
    return np.isnan(block_values.values).sum()

### Run minimize de scipy.optimize

In [30]:
extended_logs = True
constrained_cols = {}
unknown_values = np.isnan(blocks_values_simplified.values).sum()
x = np.zeros(unknown_values)
nans_location = {}

# Main function
def fill_nans(block_values,ageb_values):
    # Cantidad de valores desconocidos
    print(f"Valores desconocidos: {np.isnan(block_values.values).sum()}")
    unknown_values = np.isnan(block_values.values).sum()
    # Initial guess - El número de elementos en initial_guess debe corresponder al número de variables de decisión en tu problema de optimización.
    initial_guess = np.zeros(unknown_values)
    x = initial_guess.copy()
    
    # Constraints
    row_constraints, nan_i = create_row_constraints(block_values, nan_i=0)
    col_constraints = create_col_constraints(block_values, ageb_values, constrained_cols)
    constraints = row_constraints + col_constraints
    print(f"Constrained cols dicc: {constrained_cols}.")
    
    # Resolver el problema de optimización
    print("Starting optimization using minimize.")
    
    #return row_constraints, col_constraints
    
    resultado = minimize(objective_function, 
                         initial_guess, 
                         args=(block_values,),
                         constraints=constraints,
                         bounds=[(0, None)] * (unknown_values))
    print(resultado)

    # Preparación para remplazar nans
    filled_blocks = block_values.copy()
    # Reemplazar NaN con los valores óptimos encontrados

    print(nans_location)
    for i in nans_location.keys():
        # Find unknown value location
        index = nans_location[i][0]
        col = nans_location[i][1]
        
        print(f"nan found on Index:{index}, Col:{col}.") #Verification log
        
        # Replace unknown value in location
        filled_blocks.loc[index,col] = list(resultado.x)[i]
        
        print(f"Found value: {list(resultado.x)[i]}") #Verification log

    # Unir las columnas llenas con el resto de las columnas
    #filled_blocks_full = pd.concat([filled_blocks, block_values.drop(relevant_columns, axis=1)], axis=1)

    return filled_blocks
    
filled_blocks = fill_nans(blocks_values_simplified,ageb_values_simplified)
filled_blocks

Valores desconocidos: 6
----------------------------------------
Row 0 Step 1: Variables: ['P_0A2_tot', 'P_0A2_F', 'P_0A2_M'] Values: [4.0, 4.0, 0.0].
Row 0 Step 2: Skipped as it has no nan values in currently examined columns.
----------------------------------------
Row 1 Step 1: Variables: ['P_0A2_tot', 'P_0A2_F', 'P_0A2_M'] Values: [nan, 5.0, 2.0].
Row 1 Step 2: Nans found. Proceeding to creating constraint equation with initial nan_id 0.
Reading row 1's equation P_0A2_tot - (P_0A2_F + P_0A2_M) with original vals [nan, 5.0, 2.0].
Row 1 Step 3: Created computable row equation: x[0] - (5.0 + 2.0) by replacing 1 nans as x[i] unknowns.
Row 1 Step 3: Finishing row with nan_i: 1.
Row 1 Step 3: Finished creating constraint equation.
Row 1 Step 4: Constraints dicc: {'type': 'eq', 'fun': <function row_computable_equation at 0x7fae638b9b80>, 'args': (1, [nan, 5.0, 2.0], 0)}.
Finished row. Updated global nan_i to 1.
----------------------------------------
Row 2 Step 1: Variables: ['P_0A2_tot

,P_0A2_tot,P_0A2_F,P_0A2_M
0,4.0,4.0,0.0
1,0.0,5.0,2.0
2,3.0,0.0,2.0
3,4.0,3.0,0.0
4,0.0,0.0,0.0
